# <center>Methods of Applied Mathematics Team Project

In [3]:
from itertools import combinations
import datetime
import numpy as np
import pandas as pd
import networkx as nx
from scipy import spatial
import matplotlib.pyplot as plt
from googletrans import Translator
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
%matplotlib inline

c:\users\wnsvy\anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [4]:
stock_data = pd.read_csv('stock_price.csv', index_col=0)
stock_data.index = pd.Series(stock_data.index).apply(lambda x : datetime.datetime.strptime(x, "%Y-%m-%d"))
stock_data.head()

,A000270,A003490,A035720,A051910,A068270
date,,,,,
2017-06-07,38850.0,37400.0,108425.0,290500.0,96910.4
2017-06-08,39150.0,37500.0,108927.0,289500.0,97793.2
2017-06-09,39000.0,37950.0,110232.0,292500.0,99460.6
2017-06-12,38950.0,37200.0,105413.0,289500.0,98087.4
2017-06-13,39200.0,37300.0,107321.0,292500.0,97989.3


In [5]:
def network_applied(code, slice_data):
    
    stock_data_date = slice_data.index
    stock_data_title_trans = slice_data['en_title']
    stock_data_title_word = {}
    stock_data_cent = []
    stock_sentiment = {}
    result = 0

    for i in range(len(stock_data_title_trans)):
        text = re.sub("[^\w]", " ", stock_data_title_trans[i]).split()
        stock_data_title_word[i] = text
    
    node = []
    for i in range(len(stock_data_title_trans)):
        node += stock_data_title_word[i]
    
    link = []
    for i in range(len(stock_data_title_trans)):
        for s in stock_data_title_word[i]:
            for t in stock_data_title_word[i]:
                if s != t:
                    link += [(s,t)]
    
    G = nx.Graph()
    G.add_nodes_from(node)
    G.add_edges_from(link)
    try:
        stock_data_cent += [dict(nx.eigenvector_centrality(G))]
    except:
        return 0
    
    for w in node:
        ss=sid.polarity_scores(w)
        stock_sentiment[w]=ss['compound']
        
    c=0
    for i in node:
        c+=1
        result += stock_sentiment[i]*stock_data_cent[0][i]
    result = result / c
    
    return result

In [6]:
stock_list = ['000270','003490','035720','051910','068270']

In [13]:
def weekly_result(code,week_day):
    
    article = pd.read_csv('translated_' + code + '.csv',index_col=0)
    article.index = pd.Series(article.index).apply(lambda x : datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
    
    # 해당종목의 가격
    close = stock_data['A' + code]
    score_df = pd.DataFrame(close)
    score_df['score'] = 0
    score_df['weekday'] = list(pd.Series(close.index).apply(lambda x : x.weekday()))
    weekly_data = score_df[score_df['weekday']==week_day]
    
    sl=[]
    for i in np.arange(len(weekly_data.index[1:]))+1:
        score = 0
        start_time = weekly_data.index[i-1] + pd.Timedelta('15 hours 30 min')
        start_time = start_time.strftime('%Y-%m-%d %H:%M:%S')
        end_time = weekly_data.index[i] + pd.Timedelta('15 hours 30 min')
        end_time = end_time.strftime('%Y-%m-%d %H:%M:%S')
        
        title_data = article[start_time : end_time]
        score = network_applied(code, title_data)
        sl.append(network_applied(code, title_data))
        

    
    weekly_data['score'].iloc[1:] = sl
    
    return get_accuracy(weekly_data.iloc[:,0],weekly_data.iloc[:,1])

In [16]:
for i in stock_list:
    print('Weekly Result for ',i)
    rate = 0
    for j in range(5):
        score = weekly_result(i,j)
        print(score)
        rate += score
    print('Average Prediction Percentage : ',rate/5)

Weekly Result for  000270


c:\users\wnsvy\anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
c:\users\wnsvy\anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0.48936170212765956
0.5833333333333334
0.5294117647058824
0.6326530612244898
0.5510204081632653
Average Prediction Percentage :  0.5571560539109262
Weekly Result for  003490
0.48936170212765956
0.4583333333333333
0.5882352941176471
0.5918367346938775
0.4489795918367347
Average Prediction Percentage :  0.5153493312218504
Weekly Result for  035720
0.574468085106383
0.5416666666666666
0.5686274509803921
0.4897959183673469
0.46938775510204084
Average Prediction Percentage :  0.5287891752445659
Weekly Result for  051910
0.5957446808510638
0.5416666666666666
0.5490196078431373
0.4489795918367347
0.5306122448979592
Average Prediction Percentage :  0.5332045584191123
Weekly Result for  068270
0.6170212765957447
0.5833333333333334
0.5098039215686274
0.5918367346938775
0.5714285714285714
Average Prediction Percentage :  0.5746847675240309


In [46]:
result_df = pd.DataFrame(close)
result_df['score'] = 0
result_df['score'].iloc[1:] = sl

c:\users\wnsvy\anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,A003490,score
date,,
2017-06-07,37400.0,0.000000e+00
2017-06-08,37500.0,3.581407e-03
2017-06-09,37950.0,7.954243e-04
2017-06-12,37200.0,-1.376477e-02
2017-06-13,37300.0,0.000000e+00
2017-06-14,37700.0,3.891287e-02
2017-06-15,37200.0,7.422186e-03
2017-06-16,36850.0,9.109925e-03
2017-06-19,38750.0,2.621411e-03


In [15]:
def get_up_down(data):
    up_down = data.where(data>0)
    up_down[data.where(data>0).dropna().index] = 1
    up_down = up_down.fillna(-1)
    return up_down
    
def get_accuracy(data1,data2):
    up_down1 = get_up_down(data1.pct_change())
    # up_down1 = data1
    up_down2 = get_up_down(data2)
    
    valid = up_down1 + up_down2
    accuracy = len(valid.where(valid!=0).dropna()) / len(valid)
    return accuracy

0.4918032786885246